In [5]:
from keras_retinanet.models import load_model


In [23]:
model_keras = load_model('snapshots/resnet50_csv_50.h5', backbone_name='resnet50')


In [11]:
import tensorflow as tf


In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()


In [10]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [11]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
tflite_quant_model = converter.convert()

ValueError: For full integer quantization, a `representative_dataset` must be specified.

In [17]:
with open('model.tflitef16', 'wb') as f:
  f.write(tflite_model)

## TFITE 16 8

In [1]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib




2023-11-20 18:35:48.250271: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 18:35:48.250337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 18:35:48.301177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 18:35:48.415054: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 18:35:49.814558: W tensorflow/compiler/tf2

In [2]:
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [6]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

NameError: name 'model' is not defined

In [12]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# use this to change which GPU to use
gpu = '0'

# set the modified tf session as backend in keras
setup_gpu(gpu)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Physical devices cannot be modified after being initialized
1 Physical GPUs, 1 Logical GPUs


In [17]:

# configuration parameters 
TEST_DATA_DIR = 'PyAVE_cmu_fall2023/cmu_fall2023/images/validation_image_cmu/'
MODEL_PATH = "GHL_Models/resnet50_csv_26.h5"
TFLITE_MODEL_DIR = "./models/tflite/"
TEST_SAMPLES = 1
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [18]:
# create a directory to save the tflite models if it does not exists
if not os.path.exists(TFLITE_MODEL_DIR):
    os.makedirs(TFLITE_MODEL_DIR)

In [19]:
model = load_model('GHL_Models/resnet50_csv_26.h5', backbone_name='resnet50')


In [13]:
import argparse
import os
import sys



from keras_retinanet.preprocessing.csv_generator import CSVGenerator



def create_generator():
    """ Create generators for evaluation.
    """
    common_args = {
        
        'preprocess_image' : preprocess_image,
        
    }

    
    validation_generator = CSVGenerator(
           'val_anots.csv',
            'class.csv',
            shuffle_groups=False,
            **common_args
        )
    
    return validation_generator



In [15]:
gen = create_generator()
# create an image generator with a batch size of 1 
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_generator = gen
def represent_data_gen():
    """ it yields an image one by one """
    for ind in range(len(gen.image_data.keys())):
        img = test_generator.load_image(ind) # it returns (image and label) tuple
        img, scale = resize_image(img)
        img = np.expand_dims(img, axis=0)
        print(img.shape)
        yield [np.array(img, dtype=np.float32, ndmin=2)] # return only image

In [20]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpo8uygoft/assets


INFO:tensorflow:Assets written to: /tmp/tmpo8uygoft/assets
/home/developer/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-11-20 18:46:22.858896: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 18:46:22.859430: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-20 18:46:22.860514: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpo8uygoft
2023-11-20 18:46:22.895341: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-20 18:46:22.895372: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpo8uygoft
2023-11-20 18:46:22.992613: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-20 18:46:24

(1, 800, 1067, 3)
(1, 1201, 800, 3)
(1, 800, 1015, 3)
(1, 800, 1067, 3)
(1, 800, 926, 3)
(1, 800, 1201, 3)
(1, 800, 1102, 3)
(1, 800, 1201, 3)
(1, 1067, 800, 3)
(1, 800, 1000, 3)
(1, 800, 1061, 3)
(1, 1205, 800, 3)
(1, 800, 1201, 3)
(1, 800, 980, 3)
(1, 800, 1067, 3)
(1, 800, 1294, 3)
(1, 1067, 800, 3)
(1, 1067, 800, 3)
(1, 800, 1250, 3)
(1, 800, 1067, 3)
(1, 800, 1194, 3)
(1, 800, 1067, 3)
(1, 800, 1067, 3)
(1, 800, 1067, 3)
(1, 800, 877, 3)
(1, 800, 1170, 3)
(1, 800, 1067, 3)
(1, 1120, 800, 3)
(1, 1201, 800, 3)
(1, 800, 808, 3)


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


In [22]:
tflite_model_quant_file = tflite_models_dir/"/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/quantized_models/full_int8.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

37381296

In [25]:
ret_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=ret_model,gpu_compatibility=True)


INFO:tensorflow:Assets written to: /tmp/tmpqkj5mc66/assets


INFO:tensorflow:Assets written to: /tmp/tmpqkj5mc66/assets
2023-11-20 19:31:28.282939: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 19:31:28.282961: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-20 19:31:28.283136: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpqkj5mc66
2023-11-20 19:31:28.308058: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-20 19:31:28.308099: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpqkj5mc66
2023-11-20 19:31:28.384684: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-20 19:31:29.449414: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpqkj5mc66
2023-11-20 19:31:29.732068: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #72 and tensor #1 as input and produces tensor #151 as output.

Subgraph#0 main(T#0) -> [T#354, T#344]
  Op#0 CONV_2D(T#0, T#72, T#1) -> [T#151]
  Op#1 MAX_POOL_2D(T#151) -> [T#152]
  Op#2 CONV_2D(T#152, T#73, T#42) -> [T#153]
  Op#3 CONV_2D(T#152, T#74, T#2) -> [T#154]
  Op#4 PAD(T#154, T#149[0, 0, 1, 1, 1, ...]) -> [T#155]
  Op#5 CONV_2D(T#155, T#75, T#3) -> [T#156]
  Op#6 CONV_2D(T#156, T#76, T#43) -> [T#157]
  Op#7 ADD(T#157, T#153) -> [T#158]
  Op#8 CONV_2D(T#158, T#77, T#4) -> [T#159]
  Op#9 PAD(T#159, T#149[0, 0, 1, 1, 1, ...]) -> [T#160]
  Op#10 CONV_2D(T#160, T#78, T#5) -> [T#161]
  Op#11 CONV_2D(T#161, T#79, T#44) -> [T#162]
  Op#12 ADD(T#162, T#158) -> [T#163]
  Op#13 CONV_2D(T#163, T#80, T#6) -> [T#164]
  Op#14 PAD(T#164, T#149[0, 0, 1, 1, 1, ...]) -> [T#165]
  Op#15 CO

In [62]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen
tflite_16x8_model = converter.convert()
tflite_model_16x8_file = tflite_models_dir/"avl_model_quant_16x8.tflite"
tflite_model_16x8_file.write_bytes(tflite_16x8_model)

INFO:tensorflow:Assets written to: /tmp/tmp1kju2p8l/assets


INFO:tensorflow:Assets written to: /tmp/tmp1kju2p8l/assets
/home/developer/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


: 

: 

In [28]:
with open('model.tflite816', 'wb') as f:
  f.write(tflite_16x8_model)

In [ ]:
model = load_model('GHL_Models/resnet50_csv_26.h5', backbone_name='resnet50')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


In [22]:
tflite_models_dir = pathlib.Path("/tmp/avl_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [23]:
tflite_model_file = tflite_models_dir/"avl_model.tflite"
tflite_model_file.write_bytes(tflite_model)

145372884

In [29]:
image = tf.io.read_file('PyAVE_cmu_fall2023/cmu_fall2023/images/validation_image_cmu/10.6.3.9 Flat margin.jpg')

In [31]:
from keras_retinanet.keras_retinanet.models import load_model
from keras_retinanet.keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image

model = load_model('GHL_Models/resnet50_csv_26.h5', backbone_name='resnet50')
image = read_image_bgr('PyAVE_cmu_fall2023/cmu_fall2023/images/validation_image_cmu/10.6.3.9 Flat margin.jpg')

# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

boxes, scores = model.predict_on_batch(np.expand_dims(image, axis=0))
print(boxes, scores)

In [33]:
len(scores)

array([[[0.00081564, 0.00078893],
        [0.00068027, 0.00076989],
        [0.00073375, 0.00084641],
        ...,
        [0.00160126, 0.00288163],
        [0.0016234 , 0.00201132],
        [0.00080883, 0.00125047]]], dtype=float32)

In [58]:

mnist_ds = tf.data.Dataset.from_tensor_slices((tf.expand_dims(image, axis=0),)).batch(1)
def representative_data_gen():
    for input_value in mnist_ds.take(100):
        # Model has only one input so each data point has one element.
        yield [input_value]
converter.representative_dataset = representative_data_gen
tflite_16x8_model = converter.convert()
tflite_model_16x8_file = tflite_models_dir/"avl_model_quant_16x8.tflite"
tflite_model_16x8_file.write_bytes(tflite_16x8_model)


INFO:tensorflow:Assets written to: /tmp/tmp3kr24sjp/assets


INFO:tensorflow:Assets written to: /tmp/tmp3kr24sjp/assets
2023-11-13 17:40:33.580996: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-13 17:40:33.581021: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-13 17:40:33.581269: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp3kr24sjp
2023-11-13 17:40:33.610458: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-11-13 17:40:33.610502: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp3kr24sjp
2023-11-13 17:40:33.707206: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-13 17:40:34.787665: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp3kr24sjp
2023-11-13 17:40:35.082260: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

AttributeError: 'tuple' object has no attribute 'shape'

In [45]:


converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]


In [33]:
images = tf.cast(image, tf.float32) / 255.0

In [36]:
repr_dataset = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
    
    yield [repr_dataset.take(1)]

<_TakeDataset element_spec=TensorSpec(shape=(None, 1067, 3), dtype=tf.float32, name=None)>

In [42]:

converter.representative_dataset = representative_data_gen

In [43]:
model()

INFO:tensorflow:Assets written to: /tmp/tmp6xab3ws8/assets


INFO:tensorflow:Assets written to: /tmp/tmp6xab3ws8/assets
2023-11-13 17:28:42.176619: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-13 17:28:42.176645: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-13 17:28:42.178758: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp6xab3ws8
2023-11-13 17:28:42.223442: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-11-13 17:28:42.223485: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp6xab3ws8
2023-11-13 17:28:42.354850: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-13 17:28:43.838328: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp6xab3ws8
2023-11-13 17:28:44.173368: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
! ls -lh {tflite_models_dir}

In [35]:
interpreter = tf.lite.Interpreter(model_path=str('model.tflite816'))
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [43]:

image = preprocess_image(image)
image, scale = resize_image(image)
image.shape

(1202, 800, 3)

In [45]:
# test_image = np.expand_dims(image, axis=0).astype(np.float32)
test_image.shape

(1, 1202, 800, 3)

In [41]:
image = read_image_bgr('/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/images/Cats_Test0.png')


In [39]:

image = preprocess_image(image)
image, scale = resize_image(image)
test_image = np.expand_dims(image, axis=0).astype(np.float32)
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

interpreter.set_tensor(input_index,test_image )
interpreter.invoke()
predictions = interpreter.get_tensor(output_index)

ValueError: Cannot set tensor: Dimension mismatch. Got 800 but expected 1 for dimension 1 of input 0.

In [46]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

floating_model = (input_details[0]['dtype'] == np.float32)

In [50]:
input_details

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([1, 1, 1, 3], dtype=int32),
  'shape_signature': array([-1, -1, -1,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]